Projeto inserindo dados Json da Anac em banco postgree

Passos do projeto:

1°: importar json como DF 

2°: Tratar os dados levar apenas colunas ["Numero_da_Ocorrencia", "Classificacao_da_Ocorrência", "Data_da_Ocorrencia","Municipio","UF","Regiao","Nome_do_Fabricante"]

3°: Tirar acento de nomes de colunas

4°: Criar Banco de Dados e Tabela 

5°: Configurar a conexão 

6°: Enviar dados para o Postgree

7°: Criar um Delete da tabela para nao ter histórico , mantendo em banco sempre os dados mais rescentes 

8°: Refaturar código (apagar o que nao vai usar)

9°: Nas Aulas estamos treinando com arquivo script estensão Ipynb (notebook)  vamos salvar como extensão PY

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
caminho_do_arquivo = r"C:\Users\edmil\OneDrive\Documentos\Cursos\Python - SGBDS - Arquivos\Origem de dados\V_OCORRENCIA_AMPLA.json"
df = pd.read_json(caminho_do_arquivo, encoding='utf-8-sig')
df.head(3)

In [ ]:
df.columns

In [ ]:
colunas = ["Numero_da_Ocorrencia", "Classificacao_da_Ocorrência", "Data_da_Ocorrencia","Municipio","UF","Regiao","Nome_do_Fabricante"]
df = df[colunas]

In [ ]:
df.rename( columns={  'Classificacao_da_Ocorrência' : 'Classificacao_da_Ocorrencia'  } ,inplace=True )

In [ ]:
df.head(10)

In [ ]:
import psycopg2

# Parâmetros de conexão
dbname   = 'python'
user     = 'postgres'
password = '12345'
host     = 'localhost'
port     = '5432' 

# Criar uma conexão
conexao = psycopg2.connect(dbname=dbname,
                        user=user,
                        password=password,
                        host=host,
                        port=port)


cursor = conexao.cursor()

#Delete base antes da carga
cursor.execute(  " delete from public.Anac  ")

#Carga de Dados
for indice,coluna_df in df.iterrows():
    cursor.execute( """   insert into Anac (     
                Numero_da_Ocorrencia, 
                Classificacao_da_Ocorrencia, 
                Data_da_Ocorrencia, 
                Municipio, 
                UF, 
                Regiao, 
                Nome_do_Fabricante
            ) VALUES (%s,%s,%s,%s,%s,%s,%s) 
                
            """ ,(
                coluna_df["Numero_da_Ocorrencia"],
                coluna_df["Classificacao_da_Ocorrencia"],
                coluna_df["Data_da_Ocorrencia"],
                coluna_df["Municipio"],
                coluna_df["UF"],
                coluna_df["Regiao"],
                coluna_df["Nome_do_Fabricante"]                            
                
            )                   
            
            
            )

conexao.commit() 
cursor.close()
conexao.close()